In [0]:
%pip install chromadb==0.5.3
# %pip install chromadb --upgrade
%pip install --upgrade pydantic
%pip install --upgrade langchain_openai

In [0]:
dbutils.library.restartPython()

In [0]:
import os
# import docx
# from unstructured.partition.docx import partition_docx
import pandas as pd
from pyspark.sql.types import StructType, StructField, StringType
import json
from pyspark.sql import SparkSession
import os
import re
import requests
import sys
# from num2words import num2words
import os
import pandas as pd
import numpy as np
# import tiktoken
from openai import AzureOpenAI
import mlflow
import mlflow.deployments

# # OpenAIの設定
# os.environ["AZURE_OPENAI_API_KEY"] = dbutils.secrets.get("demo-token-ka.abe", "openai_api_key")
# os.environ["OPENAI_API_TYPE"] = "azure"
# os.environ["AZURE_OPENAI_ENDPOINT"] = dbutils.secrets.get("demo-token-ka.abe", "openai_endpoint")
# os.environ["OPENAI_API_VERSION"] = "2024-06-01"

# from openai import AzureOpenAI
# # embedding mdoelを読み込む
# client = AzureOpenAI(
#   api_key = os.getenv("Azure_OPENAI_API_KEY"),  
#   api_version = os.getenv("OPENAI_API_VERSION"),
#   azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
# )

client = mlflow.deployments.get_deploy_client("databricks")

def generate_embeddings(text, model="text-embedding-3-small"):
    return client.predict(endpoint="databricks-bge-large-en", inputs={"input": [text]})

In [0]:
import pandas as pd
# df = spark.read.table("world_tour_2024.dbdemos_rag_chatbot.databricks_documentation_embedding")
df = spark.read.table("apps_data.dbx_doc.databricks_doc")
pandas_df = df.toPandas()

In [0]:
# import chromadb
# import json
# from langchain_openai import AzureOpenAIEmbeddings

# # chroma_client = chromadb.Client()
# chroma_client = chromadb.Client()

# collection = chroma_client.create_collection(name = "dbx_collection")

In [0]:
import chromadb
import json
from chromadb.config import Settings

chroma_client = chromadb.PersistentClient(
    path="/Volumes/apps_data/dbx_doc/embedding/",
    settings=Settings(anonymized_telemetry=False)
)

In [0]:
# returns a list of the first 10 items in the collection
collection.peek()

In [0]:
# returns the number of items in the collection
collection.count()

In [0]:
# クエリテキスト: 初期段階では、特許の要約部分を取得
query_text = "how to create the delta live table"
# query_text = "デルタライブテーブルをどのように作成しますか。"

# クエリテキストの埋め込みを生成
query_embedding = generate_embeddings(query_text)

# コレクションをクエリして最も類似したドキュメントを取得
results = collection.query(
    query_embeddings=query_embedding,
    n_results=10  # 取得する結果の数を指定
)

# 結果を表示
ids_list = []
for result_list in results['ids']:
    for result in result_list:
        ids_list.append(int(result)) 
